In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm
from scipy.stats import wasserstein_distance
import time

In [781]:
def load_model(path):
    with np.load(path) as data:
        pi = data["pi"]
        A = data["A"]
        B = data["B"]
        param = data["param"]
        bins_z = data["bins_z"]
        bins_co2 = data["bins_co2"]
        
    return pi, A, B, param, bins_z, bins_co2


def forward(x,pi,A,b):
    """
    To calculate scaled forward variables 
    Returns: alphaHat = P(S_t=i|x) probability of a state i at time instance t given observations x
    x: observations (T, )
    pi: initial state probability (N, )
    A: Transition probability matrix (N,N)
    b: emission probability matrix (N, len(np.unique(observations)))
    """
    N = np.shape(b)[0]
    T = np.shape(x)[0]
    alphaTemp = np.zeros((N,T))
    alphaHat = np.zeros((N,T))
    c = np.zeros(T)

    #Initialization
    alphaTemp[:,0] = pi*b[:,x[0]]
    c[0] = np.sum(alphaTemp[:,0])
    alphaHat[:,0]=alphaTemp[:,0]/c[0]
    #print(alphaTemp[:,0])
    #Forward step
    for t in range (1,T):
            for j in range(N):
                alphaTemp[j,t] = b[j,x[t]]*(np.dot(alphaHat[:,t-1],A[:,j]))
                #print("alphaTemp[j,t]:",b[j,x[t]],(np.dot(alphaHat[:,t-1],A[:,j])))
            c[t] = np.sum(alphaTemp[:,t])
            #print("c[t]:",c[t])
            alphaHat[:,t]=alphaTemp[:,t]/c[t]
            #print("alphaHat[:,t]", alphaHat[:,t])
            
    return(alphaHat,c)


def sync_sequence(timestamps,data):
    start_indices = []
    for ind,t in enumerate(timestamps):
        df = data[ind]
        df['create_time'] = pd.date_range('01/01/2022 00:00:00', periods=len(df), freq='1T')
        df.loc[:, 'create_time'] = pd.to_datetime(df.loc[:, 'create_time'])
        df = df.set_index('create_time')
        data_time = np.array(df.time.resample('20T').min())

        """data_time = np.array(data[ind].time)"""

        start = np.where(data_time==t)[0][0]
        start_indices.append(start)

    return start_indices


def get_measurement_sequence(start_ind,amt,data):
    
    #data = data.drop(data.index[detect_outliers(data.lp8_T)])
    #data = data.drop(data.index[detect_outliers(data.lp8_ir)])
    #data = data.drop(data.index[detect_outliers(data.lp8_CO2)])

    #data = data.dropna()
    #data = data.drop(data.index[np.unique(np.hstack((detect_outliers(data.lp8_CO2), detect_outliers(data.S_co2), detect_outliers(data.lp8_T), detect_outliers(data.lp8_ir))))])

    data['create_time'] = pd.date_range('01/01/2022 00:00:00', periods=len(data), freq='1T')
    data.loc[:, 'create_time'] = pd.to_datetime(data.loc[:, 'create_time'])
    data = data.set_index('create_time')
    data_time = np.array(data.time.resample('20T').min())[start_ind:start_ind+amt]
    ref_meas = np.array(data.S_co2.resample('20T').mean())[start_ind:start_ind+amt]
    #ref_temp = np.array(data[5].resample('16T').mean())
    data_CO2 = np.array(data.lp8_CO2.resample('20T').mean())[start_ind:start_ind+amt]
    data_temp = np.array(data.lp8_T.resample('20T').mean())[start_ind:start_ind+amt]
    data_ir = np.array(data.lp8_ir.resample('20T').mean())[start_ind:start_ind+amt]

    """
    data_time = np.array(data.time)[start_ind:start_ind+amt*20:20]
    data_temp = np.array(data.lp8_T)[start_ind:start_ind+amt*20:20]
    data_ir = np.array(data.lp8_ir)[start_ind:start_ind+amt*20:20]
    data_CO2 = np.array(data.lp8_CO2)[start_ind:start_ind+amt*20:20]
    ref_meas = np.array(data.S_co2)[start_ind:start_ind+amt*20:20]
    """
    
    print(data_time[0],data_time[-1])

    return data_temp,data_ir,data_CO2,ref_meas,data_time


def state_to_value(states,bins):
    zeros = np.zeros_like(states)
    itvl = bins[-1] - bins[-2]
    for i in range(len(states)):
        if i == 0:
            zeros[states == i] = bins[i] - itvl / 2     
        elif i == len(bins):
            zeros[states == i] = bins[-1] + itvl / 2
        else:
            zeros[states == i] = (bins[i] + bins[i - 1]) / 2

    return zeros


def calc_co2(zero, ir, temp, s, t0, tz, tz2, ts, ts2,fp):

    """
    Mapping sensor measurements to concentration level of CO2 based on Beer-Lambert Law
    :param zero: zero calibration parameter
    :param ir: amount of IR light received by detector
    :param temp: measured environmental temperature
    :param s: span calibration parameter
    :param t0: full transmittance (scaled and shifted transmittance when the target concentration is zero)
    :param tz: temperature calibration parameter, describes the temperature dependency of zero
    :param tz2: temperature calibration parameter, describes the temperature dependency of zero
    :param ts: temperature calibration parameter, describes the temperature dependency of span
    :param ts2: temperature calibration parameter, describes the temperature dependency of span
    """

    '''
    The formula is owned by the company.
    '''

    return co2


def viterbi_log(y, pi, A, B):

    """
    Viterbi Algorithm
    :param y: observations (N, )
    :param pi: state probability (S, )
    :param A: transition probability matrix (S, S)
    :param B: observation probability matrix (S, N)
    :return: predicted state sequence (N, )
    """

    S = A.shape[0]    # Number of states
    N = len(y)  # Length of observation sequence
    tiny = np.finfo(0.).tiny
    pi_log = np.log(pi + tiny)
    A_log = np.log(A + tiny)
    B_log = np.log(B + tiny)

    # Initialization
    ### Viterbi probability vector
    D_log = np.zeros((S, N))
    D_log[:, 0] = pi_log + B_log[:, y[0]]
    ### Viterbi backpointer matrix
    E = np.zeros((S, N - 1)).astype(np.int32)

    # Viterbi forward step
    for n in range(1, N):
        for i in range(S):
            temp_sum = A_log[:, i] + D_log[:, n - 1]
            D_log[i, n] = np.max(temp_sum) + B_log[i, y[n]]
            E[i, n - 1] = np.argmax(temp_sum)

    # Backtracking
    S_opt = np.zeros(N).astype(np.int32)
    S_opt[-1] = np.argmax(D_log[:, -1])
    for n in range(N - 2, -1, -1):
        S_opt[n] = E[int(S_opt[n + 1]), n]

    return S_opt


def sensor_fusion_dsrule(p):
    fused_belief = np.prod(p,axis=1)
    if np.sum(fused_belief) != 0:
        fused_belief = fused_belief/np.sum(fused_belief)
    return fused_belief


def sensor_fusion_wavg(p):
    #Wasserstein distance
    I = p.shape[1]
    dist = np.zeros((I,I))
    for i in range(I):
        for j in range(I):
            dist[i,j] = wasserstein_distance(p[:,i],p[:,j])
    #Normalize distances
    n_dist = 2*dist/np.sum(dist)

    #Similarity between belief functions
    s = 1-n_dist

    #Support degrees, i-th element is support degree of belief function P_i
    supp_deg = np.sum(s,axis = 0)
    #Weighting factors
    weights = supp_deg/np.sum(supp_deg)

    #Weighted average of I belief functions
    P_hat = np.sum(weights*p, axis = 1)

    #DS rule to combine P_hat
    fused_belief = np.zeros_like(P_hat)
    norm_factor = 0
    for k in range(P_hat.shape[0]):
        fused_belief[k] = P_hat[k]**(I)
        norm_factor += P_hat[k]**(I)
    fused_belief = fused_belief/norm_factor

    return fused_belief

In [ ]:
start = time.time()

pi_0, A_0, B_0, param_0, bins_z_0, bins_co2_0 = load_model("sensor_fusion/1214_5_days_diff/1214_LP8_Supervised_HMM_Sensor_00.npz")
pi_1, A_1, B_1, param_1, bins_z_1, bins_co2_1 = load_model("sensor_fusion/1214_5_days_diff/1214_LP8_Supervised_HMM_Sensor_01.npz")
pi_2, A_2, B_2, param_2, bins_z_2, bins_co2_2 = load_model("sensor_fusion/1214_5_days_diff/1214_LP8_Supervised_HMM_Sensor_02.npz")
pi_3, A_3, B_3, param_3, bins_z_3, bins_co2_3 = load_model("sensor_fusion/1214_5_days_diff/1214_LP8_Supervised_HMM_Sensor_03.npz")
pi_4, A_4, B_4, param_4, bins_z_4, bins_co2_4 = load_model("sensor_fusion/1214_5_days_diff/1214_LP8_Supervised_HMM_Sensor_04.npz")

#No. of states for each HMM
N_0 = pi_0.shape[0]
N_1 = pi_1.shape[0]
N_2 = pi_2.shape[0]
N_3 = pi_3.shape[0]
N_4 = pi_4.shape[0]

#Map each state with a corresponding zero value
zeros_0 = state_to_value(np.arange(N_0),bins_z_0)
zeros_1 = state_to_value(np.arange(N_1),bins_z_1)
zeros_2 = state_to_value(np.arange(N_2),bins_z_2)
zeros_3 = state_to_value(np.arange(N_3),bins_z_3)
zeros_4 = state_to_value(np.arange(N_4),bins_z_4)

data0 = pd.read_csv("data/Measurements/5_days_diff/kth_logger_00.csv", delimiter=',')
data1 = pd.read_csv("data/Measurements/5_days_diff/kth_logger_01.csv", delimiter=',')
data2 = pd.read_csv("data/Measurements/5_days_diff/kth_logger_02.csv", delimiter=',')
data3 = pd.read_csv("data/Measurements/5_days_diff/kth_logger_03.csv", delimiter=',')
data4 = pd.read_csv("data/Measurements/5_days_diff/kth_logger_04.csv", delimiter=',')

start_timestamps = ["2022-12-05 21:04:59","2022-12-05 20:56:41","2022-12-05 21:02:28","2022-12-05 21:03:46","2022-12-05 21:07:18"]
data = [data0, data1, data2, data3, data4]
start_i = sync_sequence(start_timestamps,data)

L = 95

fused_co2 = np.zeros(L)
refs = np.zeros((L))
pre_co2s = np.zeros((L,5))

#Observe the L measurements
data_temp_0,data_ir_0,data_CO2_0,ref0,data_time_0 = get_measurement_sequence(start_i[0],L,data0)
data_temp_1,data_ir_1,data_CO2_1,ref1,data_time_1 = get_measurement_sequence(start_i[1],L,data1)
data_temp_2,data_ir_2,data_CO2_2,ref2,_ = get_measurement_sequence(start_i[2],L,data2)
data_temp_3,data_ir_3,data_CO2_3,ref3,_ = get_measurement_sequence(start_i[3],L,data3)
data_temp_4,data_ir_4,data_CO2_4,ref4,_ =  get_measurement_sequence(start_i[4],L,data4)


all_refs = np.stack((ref0,ref1,ref2,ref3,ref4),axis = 1)

x0 = np.digitize(data_CO2_0,bins_co2_0)
x1 = np.digitize(data_CO2_1,bins_co2_1)
x2 = np.digitize(data_CO2_2,bins_co2_2)
x3 = np.digitize(data_CO2_3,bins_co2_3)
x4 = np.digitize(data_CO2_4,bins_co2_4)

### Forward Algorithm

In [ ]:
#Calculate belief functions
A_0 = (A_0 + 1e-10)/np.sum(A_0 + 1e-10,axis=1)
beliefs_0,_ = forward(x0,pi_0,A_0,B_0)
A_1 = (A_1 + 1e-10)/np.sum(A_1 + 1e-10,axis=1)
beliefs_1,_ = forward(x1,pi_1,A_1,B_1)
A_2 = (A_2 + 1e-10)/np.sum(A_2 + 1e-10,axis=1)
beliefs_2,_ = forward(x2,pi_2,A_2,B_2)
A_3 = (A_3 + 1e-10)/np.sum(A_3 + 1e-10,axis=1)
beliefs_3,_ = forward(x3,pi_3,A_3,B_3)
A_4 = (A_4 + 1e-10)/np.sum(A_4 + 1e-10,axis=1)
beliefs_4,_ = forward(x4,pi_4,A_4,B_4)

#For plotting purposes, predicted co2 for each sensor
pre_state_0 = viterbi_log(x0,pi_0,A_0,B_0)
pre_state_1 = viterbi_log(x1,pi_1,A_1,B_1)
pre_state_2 = viterbi_log(x2,pi_2,A_2,B_2)
pre_state_3 = viterbi_log(x3,pi_3,A_3,B_3)
pre_state_4 = viterbi_log(x4,pi_4,A_4,B_4)

for i in range(L):
    #Map distribution of zeros into co2 
    co2_values_0 = calc_co2(zeros_0,data_ir_0[i]*2**(-8),data_temp_0[i],param_0[5],61440,param_0[0],param_0[1],param_0[2],param_0[3],param_0[6:])
    co2_values_1 = calc_co2(zeros_1,data_ir_1[i]*2**(-8),data_temp_1[i],param_1[5],61440,param_1[0],param_1[1],param_1[2],param_1[3],param_1[6:])
    co2_values_2 = calc_co2(zeros_2,data_ir_2[i]*2**(-8),data_temp_2[i],param_2[5],61440,param_2[0],param_2[1],param_2[2],param_2[3],param_2[6:])
    co2_values_3 = calc_co2(zeros_3,data_ir_3[i]*2**(-8),data_temp_3[i],param_3[5],61440,param_3[0],param_3[1],param_3[2],param_3[3],param_3[6:])
    co2_values_4 = calc_co2(zeros_4,data_ir_4[i]*2**(-8),data_temp_4[i],param_4[5],61440,param_4[0],param_4[1],param_4[2],param_4[3],param_4[6:])
    
    if i == 25:
        #Plot belief functions
        fig, axs = plt.subplots(nrows=5, ncols=1, sharex=True,sharey=True)
        axs[0].plot(co2_values_0,beliefs_0[:,i])
        axs[1].plot(co2_values_1,beliefs_1[:,i])
        axs[2].plot(co2_values_2,beliefs_2[:,i])
        axs[3].plot(co2_values_3,beliefs_3[:,i])
        axs[4].plot(co2_values_4,beliefs_4[:,i])
        fig.suptitle("Belief Functions")
        fig.supylabel('Probability')
        fig.supxlabel('CO2 Level in ppm')
        plt.show()
    

    #Interpolation
    all_co2 = np.concatenate((co2_values_0,co2_values_1,co2_values_2,co2_values_3,co2_values_4))
    all_co2 = np.sort(all_co2)

    belief_0_inter=np.zeros(all_co2.shape)
    belief_0_inter[np.searchsorted(all_co2,co2_values_0)]=beliefs_0[:,i]
    belief_1_inter = np.zeros(all_co2.shape)
    belief_1_inter[np.searchsorted(all_co2,co2_values_1)]=beliefs_1[:,i]
    belief_2_inter = np.zeros(all_co2.shape)
    belief_2_inter[np.searchsorted(all_co2,co2_values_2)]=beliefs_2[:,i]
    belief_3_inter = np.zeros(all_co2.shape)
    belief_3_inter[np.searchsorted(all_co2,co2_values_3)]=beliefs_3[:,i]
    belief_4_inter = np.zeros(all_co2.shape)
    belief_4_inter[np.searchsorted(all_co2,co2_values_4)]=beliefs_4[:,i]
    
    
    if i == 25:
        fig, axs = plt.subplots(nrows=5, ncols=1, sharex=True,sharey=True)
        axs[0].plot(all_co2,belief_0_inter)
        axs[1].plot(all_co2,belief_1_inter)
        axs[2].plot(all_co2,belief_2_inter)
        axs[3].plot(all_co2,belief_3_inter)
        axs[4].plot(all_co2,belief_4_inter)
        fig.suptitle("Belief Functions, interpolated")
        fig.supylabel('Probability')
        fig.supxlabel('CO2 Level in ppm')
        plt.show()
    
    #Stack all belief functions into one
    beliefs = np.stack((belief_0_inter,belief_1_inter,belief_2_inter,belief_3_inter,belief_4_inter),axis=1)

    #Belief fusion
    fused_DS = sensor_fusion_dsrule(beliefs)
    fused_wavg = sensor_fusion_wavg(beliefs)

    
    if i == 25:
        plt.plot(all_co2,fused_wavg)
        plt.title("Weighted average approach")
        plt.xlabel('CO2 Level in ppm')
        plt.ylabel('Probability')
        plt.show()


    fused_co2[i]=all_co2[np.argmax(fused_wavg)]
    
    #Plotting purposes
    pre_co2_0 = calc_co2(zeros_0[pre_state_0[i]],data_ir_0[i]*2**(-8),data_temp_0[i],param_0[5],61440,param_0[0],param_0[1],param_0[2],param_0[3],param_0[6:])
    pre_co2_1 = calc_co2(zeros_1[pre_state_1[i]],data_ir_1[i]*2**(-8),data_temp_1[i],param_1[5],61440,param_1[0],param_1[1],param_1[2],param_1[3],param_1[6:])
    pre_co2_2 = calc_co2(zeros_2[pre_state_2[i]],data_ir_2[i]*2**(-8),data_temp_2[i],param_2[5],61440,param_2[0],param_2[1],param_2[2],param_2[3],param_2[6:])
    pre_co2_3 = calc_co2(zeros_3[pre_state_3[i]],data_ir_3[i]*2**(-8),data_temp_3[i],param_3[5],61440,param_3[0],param_3[1],param_3[2],param_3[3],param_3[6:])
    pre_co2_4 = calc_co2(zeros_4[pre_state_4[i]],data_ir_4[i]*2**(-8),data_temp_4[i],param_4[5],61440,param_4[0],param_4[1],param_4[2],param_4[3],param_4[6:])
    pre_co2s[i,0] = pre_co2_0
    pre_co2s[i,1] = pre_co2_1
    pre_co2s[i,2] = pre_co2_2
    pre_co2s[i,3] = pre_co2_3
    pre_co2s[i,4] = pre_co2_4

end = time.time()

In [ ]:
plt.plot(np.arange(fused_co2.shape[0]),fused_co2,np.mean(all_refs,axis = 1))
#plt.plot(np.arange(fused_co2.shape[0]),pre_co2s)
plt.ylabel("Co2 concentration in ppm")
plt.title("Sensor fusion results, belief functions with forward algorithm")
plt.legend(labels=["Fused result","Mean of references", "Predicted co2 0", "Predicted co2 1","Predicted co2 2","Predicted co2 3","Predicted co2 4"])
#data_time = np.array(data0.time)[start_i[0]:L*20+start_i[0]]
plt.xticks(np.array([0,L-1]), [data_time_0[0], data_time_0[-1]])
plt.xlabel("Time")
plt.show()

plt.plot(np.arange(fused_co2.shape[0]),ref0, color='tab:red')
plt.plot(np.arange(fused_co2.shape[0]),ref1, color ='tab:brown')
plt.plot(np.arange(fused_co2.shape[0]),ref3, color = 'tab:purple')
#plt.plot(np.arange(fused_co2.shape[0]),fused_co2, color = 'tab:blue')
plt.legend(labels = ["ref0","ref1","ref3"])
plt.title("Reference measurements")
plt.ylabel("Co2 concentration in ppm")
plt.xticks(np.array([0,L-1]), [data_time_0[0], data_time_0[-1]])
plt.xlabel("Time")
plt.show()

plt.plot(np.arange(fused_co2.shape[0]),fused_co2)
plt.plot(np.arange(fused_co2.shape[0]),np.mean(all_refs,axis = 1),np.mean(pre_co2s,axis = 1))
plt.xticks(np.array([0,L-1]), [data_time_0[0], data_time_0[-1]])
plt.title("Sensor fusion results, belief functions with forward algorithm")
plt.legend(labels = ["Sensor fusion results", "Mean of references","Mean of predictions"])
plt.ylabel("Co2 concentration in ppm")
plt.xticks(np.array([0,L-1]), [data_time_0[0], data_time_0[-1]])
plt.xlabel("Time")
plt.show()
print(end-start)

### Viterbi Algorithm

In [ ]:
# predicted co2 for each sensor
pre_state_0 = viterbi_log(x0,pi_0,A_0,B_0)
pre_state_1 = viterbi_log(x1,pi_1,A_1,B_1)
pre_state_2 = viterbi_log(x2,pi_2,A_2,B_2)
pre_state_3 = viterbi_log(x3,pi_3,A_3,B_3)
pre_state_4 = viterbi_log(x4,pi_4,A_4,B_4)

beliefs_0 = A_0[pre_state_0,:].T[:,0:-1]
beliefs_1 = A_1[pre_state_1,:].T[:,0:-1]
beliefs_2 = A_2[pre_state_2,:].T[:,0:-1]
beliefs_3 = A_3[pre_state_3,:].T[:,0:-1]
beliefs_4 = A_4[pre_state_4,:].T[:,0:-1]

for i in range(beliefs_0.shape[1]):
    #Map distribution of zeros into co2 
    co2_values_0 = calc_co2(zeros_0,data_ir_0[i+1]*2**(-8),data_temp_0[i+1],param_0[5],61440,param_0[0],param_0[1],param_0[2],param_0[3],param_0[6:])
    co2_values_1 = calc_co2(zeros_1,data_ir_1[i+1]*2**(-8),data_temp_1[i+1],param_1[5],61440,param_1[0],param_1[1],param_1[2],param_1[3],param_1[6:])
    co2_values_2 = calc_co2(zeros_2,data_ir_2[i+1]*2**(-8),data_temp_2[i+1],param_2[5],61440,param_2[0],param_2[1],param_2[2],param_2[3],param_2[6:])
    co2_values_3 = calc_co2(zeros_3,data_ir_3[i+1]*2**(-8),data_temp_3[i+1],param_3[5],61440,param_3[0],param_3[1],param_3[2],param_3[3],param_3[6:])
    co2_values_4 = calc_co2(zeros_4,data_ir_4[i+1]*2**(-8),data_temp_4[i+1],param_4[5],61440,param_4[0],param_4[1],param_4[2],param_4[3],param_4[6:])

    if i == 25:
        #Plot belief functions
        fig, axs = plt.subplots(nrows=5, ncols=1, sharex=True,sharey=True)
        axs[0].plot(co2_values_0,beliefs_0[:,i])
        axs[1].plot(co2_values_1,beliefs_1[:,i])
        axs[2].plot(co2_values_2,beliefs_2[:,i])
        axs[3].plot(co2_values_3,beliefs_3[:,i])
        axs[4].plot(co2_values_4,beliefs_4[:,i])
        fig.suptitle("Belief Functions")
        fig.supylabel('Probability')
        fig.supxlabel('CO2 Level in ppm')
        plt.show()

    #Interpolation
    all_co2 = np.concatenate((co2_values_0,co2_values_1,co2_values_2,co2_values_3,co2_values_4))
    #all_co2 = np.concatenate((co2_values_1,co2_values_2,co2_values_3,co2_values_4))
    all_co2 = np.sort(all_co2)
    
    belief_0_inter=np.zeros(all_co2.shape)
    belief_0_inter[np.searchsorted(all_co2,co2_values_0)]=beliefs_0[:,i]
    belief_1_inter = np.zeros(all_co2.shape)
    belief_1_inter[np.searchsorted(all_co2,co2_values_1)]=beliefs_1[:,i]
    belief_2_inter = np.zeros(all_co2.shape)
    belief_2_inter[np.searchsorted(all_co2,co2_values_2)]=beliefs_2[:,i]
    belief_3_inter = np.zeros(all_co2.shape)
    belief_3_inter[np.searchsorted(all_co2,co2_values_3)]=beliefs_3[:,i]
    belief_4_inter = np.zeros(all_co2.shape)
    belief_4_inter[np.searchsorted(all_co2,co2_values_4)]=beliefs_4[:,i]

    if i == 25:
        fig, axs = plt.subplots(nrows=5, ncols=1, sharex=True,sharey=True)
        axs[0].plot(all_co2,belief_0_inter)
        axs[1].plot(all_co2,belief_1_inter)
        axs[2].plot(all_co2,belief_2_inter)
        axs[3].plot(all_co2,belief_3_inter)
        axs[4].plot(all_co2,belief_4_inter)
        fig.suptitle("Belief Functions, interpolated")
        fig.supylabel('Probability')
        fig.supxlabel('CO2 Level in ppm')
        plt.show()

    #Stack all belief functions into one
    beliefs = np.stack((belief_0_inter,belief_1_inter,belief_2_inter,belief_3_inter,belief_4_inter),axis=1)

    #Belief fusion
    fused_DS = sensor_fusion_dsrule(beliefs)
    fused_wavg = sensor_fusion_wavg(beliefs,i)

    if i == 25:
        fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True,sharey=True)
        axs[0].plot(all_co2,fused_DS)
        axs[0].set_title("Dempster's rule")
        axs[1].plot(all_co2,fused_wavg)
        axs[1].set_title("Weighted average approach")
        fig.supylabel('Probability')
        fig.supxlabel('CO2 Level in ppm')
        plt.show()

    fused_co2[i]=all_co2[np.argmax(fused_wavg)]

    #For plotting predicitions
    pre_co2_0 = calc_co2(zeros_0[pre_state_0[i+1]],data_ir_0[i+1]*2**(-8),data_temp_0[i+1],param_0[5],61440,param_0[0],param_0[1],param_0[2],param_0[3],param_0[6:])
    pre_co2_1 = calc_co2(zeros_1[pre_state_1[i+1]],data_ir_1[i+1]*2**(-8),data_temp_1[i+1],param_1[5],61440,param_1[0],param_1[1],param_1[2],param_1[3],param_1[6:])
    pre_co2_2 = calc_co2(zeros_2[pre_state_2[i+1]],data_ir_2[i+1]*2**(-8),data_temp_2[i+1],param_2[5],61440,param_2[0],param_2[1],param_2[2],param_2[3],param_2[6:])
    pre_co2_3 = calc_co2(zeros_3[pre_state_3[i+1]],data_ir_3[i+1]*2**(-8),data_temp_3[i+1],param_3[5],61440,param_3[0],param_3[1],param_3[2],param_3[3],param_3[6:])
    pre_co2_4 = calc_co2(zeros_4[pre_state_4[i+1]],data_ir_4[i+1]*2**(-8),data_temp_4[i+1],param_4[5],61440,param_4[0],param_4[1],param_4[2],param_4[3],param_4[6:])
    pre_co2s[i,0] = pre_co2_0
    pre_co2s[i,1] = pre_co2_1
    pre_co2s[i,2] = pre_co2_2
    pre_co2s[i,3] = pre_co2_3
    pre_co2s[i,4] = pre_co2_4
    
print(fused_co2.shape, pre_co2s.shape, all_refs.shape)
end = time.time()

In [ ]:
plt.plot(np.arange(fused_co2.shape[0]),fused_co2,np.mean(all_refs[1:,:],axis = 1))
plt.ylabel("Co2 concentration in ppm")
plt.title("Sensor fusion results, belief functions with viterbi algorithm")
plt.legend(labels=["Fused result","Mean of references", "Predicted co2 0", "Predicted co2 1","Predicted co2 2","Predicted co2 3","Predicted co2 4"])
plt.xticks(np.array([0,L-1]), [data_time_0[1], data_time_0[-1]])
plt.xlabel("Time")
plt.show()

plt.plot(np.arange(fused_co2.shape[0]),fused_co2)
plt.plot(np.arange(fused_co2.shape[0]),np.mean(all_refs[1:,:],axis = 1),np.mean(pre_co2s,axis = 1))
plt.xticks(np.array([0,L-1]), [data_time_0[1], data_time_0[-1]])
plt.ylabel("Co2 concentration in ppm")
plt.title("Sensor fusion results, belief functions with viterbi algorithm")
plt.legend(labels=["Fused result","Mean of references", "Mean of predictions"])
plt.xlabel("Time")
plt.show()

print(end-start)